# Backtesting ML Classification-Based

## Load the model

In [24]:
import pickle

In [25]:
with open('models/model_dt_classification.pkl', 'rb') as f:
    model_dt = pickle.load(f)

In [26]:
model_dt

DecisionTreeClassifier(max_depth=15)

## Load the data

In [27]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2016-12-08,56.325228,56.582507,55.902560,56.058762,21220800,1.549143,UP
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.321692,UP
...,...,...,...,...,...,...,...
2023-03-14,256.750000,261.070007,255.860001,260.790009,33620300,1.751806,UP
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000,3.895731,UP


## Backtesting.py Library

### Create your Strategy Class

In [46]:
from backtesting import Backtest, Strategy

In [47]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [48]:
df_explanatory.iloc[-1:, :]

,Open,High,Low,Close,Volume
Date,,,,,
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000


In [49]:
df.change_tomorrow_direction[-1:]

Date
2023-03-15    UP
Name: change_tomorrow_direction, dtype: object

In [50]:
class SimpleClassificationUD(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[-1:, :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        # conditions to sell or buy

#### Calculate Purchase Recommendation

##### Buy if it goes up

In [51]:
list_buy = []

In [52]:
for direction_tomorrow in df.change_tomorrow_direction:
    if direction_tomorrow == 'UP':
        list_buy.append(1)
    else:
        list_buy.append(0)

In [53]:
df['buy'] = list_buy

In [54]:
df

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction,buy,buy_sell
Date,,,,,,,,,
2016-12-08,56.325228,56.582507,55.902560,56.058762,21220800,1.549143,UP,1,1
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.321692,UP,1,0
...,...,...,...,...,...,...,...,...,...
2023-03-14,256.750000,261.070007,255.860001,260.790009,33620300,1.751806,UP,1,0
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000,3.895731,UP,1,0


##### Buy if it goes and sell if down

> You can only sell if you have already bought

In [55]:
list_buy_sell = []
already_bought = False

In [56]:
for direction_tomorrow in df.change_tomorrow_direction:
    if direction_tomorrow == 'UP' and already_bought == False:
        list_buy_sell.append(1)
        already_bought = True
    elif direction_tomorrow == 'DOWN' and already_bought == True:
        list_buy_sell.append(-1)
        already_bought = False
    else:
        list_buy_sell.append(0)

In [57]:
df['buy_sell'] = list_buy_sell

In [58]:
df

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction,buy,buy_sell
Date,,,,,,,,,
2016-12-08,56.325228,56.582507,55.902560,56.058762,21220800,1.549143,UP,1,1
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.321692,UP,1,0
...,...,...,...,...,...,...,...,...,...
2023-03-14,256.750000,261.070007,255.860001,260.790009,33620300,1.751806,UP,1,0
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000,3.895731,UP,1,0


#### Add conditions to the strategy

In [59]:
class SimpleClassificationUD(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[-1:, :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        # conditions to sell or buy
        if forecast_tomorrow == 'UP' and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow == 'DOWN' and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Define initial conditions

In [60]:
bt = Backtest(df_explanatory, SimpleClassificationUD,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting

In [61]:
results = bt.run()

### Interpret backtesting results

In [62]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2023-03-15 00:00:00
Duration,2288 days 00:00:00
Exposure Time [%],99.873096
Equity Final [$],38432829.251776
Equity Peak [$],38432829.251776
Return [%],384228.292518


### Visualize the strategy performance

In [63]:
bt.plot(filename='reports_backtesting/classification.html')

Row(id='2427', ...)